In [ ]:
## Distributed python packages
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import datetime

from math import radians, log
## Own python package
from source_microseism import spectrogram
__author__ = "Lisa Tomasetto"
__copyright__ = "Copyright 2024, UGA"
__credits__ = ["Lisa Tomasetto"]
__version__ = "1.0"
__maintainer__ = "Lisa Tomasetto"
__email__ = "lisa.tomasetto@univ-grenoble-alpes.fr"
__status__ = "Developpement"

# Spectrograms From WW3 Hindcast

This Jupyter Notebook aims at computing spectrograms from oceanographic hindcast WAVEWWATCHIII. This is the sequel of another Notebook entitled "Rayleigh Source of Microseisms" in which the user can save the sources of Rayleigh waves matrix.

[NOTE TO THE USER]

If you haven't run the first notebook with the ```save = True``` variable, please do before continuing. 

In [ ]:
## Path to netcdf
path_netcdf = '/Users/tomasetl/Documents/code/ocean_source/pr_oceanswell/rayleigh/SDF/'

## Dates
start_date = datetime.datetime(2008, 3, 8, 0, 0, 0)
end_date = datetime.datetime(2008, 3, 19, 0, 0, 0)
dates = np.arange(start_date, end_date, datetime.timedelta(hours=3))

## Spectrogram Computation

We compute spectrograms using equations given by Ardhuin et al. (2011).
The previous code computes the equivalent source of the power spectrum of the vertical displacement $SDF(f_s)$ in $\text{m.s}$.
From it, we compute the power spectrum of the vertical **ground** displacement at a station of latitude $\lambda$ and longitude $\phi$ in $\text{m}^2.\text{s}^{-1}$, 
$$F_{\delta}(\lambda, \phi, f_s) = \int_{-\pi/2}^{\pi/2} \int_0^{2\pi} \frac{S_{DF}(f_s)}{R_E sin\Delta} e^{-2\pi f_s\Delta R_E(UQ)}(R_E^2sin\phi'd\lambda'd\phi')$$
with
- $Q$: the dissipation quality factor
- $\Delta$: the distance between source and station
- $R_E$: Earth's Radius in meters
- $U$: the group velocity of Rayleigh waves

We plot the modeled spectrogram at each timestep of the model (3-hour resolution) as:
$$S_{spectrogram}(f_s) = 10log_{10}(\sqrt{F_{\delta}(\lambda, \phi, f_s)})$$


## Spectrogram Plot

We plot here the synthetic spectrogram using the function spectrogram.
This function has 2 mandatory arguments:
- path_to_netcdf : the path to netcdf files computed previously
- dates : date vector computed above

And 4 optional arguments:
- param lon_sta: float, optional, longitude of the station (default=-21.3268) II.BORG 
- param lat_sta: float, optional, latitude of the station (default=64.7474) II.BORG
- param Q: int, optional, attenuation factor constant (default=200)
- param U: int, optional, group velocity of Rayleigh waves constant (default=1800 m/s)
    

In [ ]:
spectrogram(path_netcdf, dates, lon_sta=-158.011 , lat_sta=21.42 , Q=580)

## Real Data Comparison

Let's compare to real data. Here we computed the spectrogram for station IU.KIP, from the 8-19 March 2008. 

In [ ]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from obspy.imaging.spectrogram import spectrogram

# Set Dates
t1 = UTCDateTime(start_date)
t2 = UTCDateTime(end_date)

# Request Data
client = Client("IRIS")
st = client.get_waveforms("IU", "KIP", "*", "LHZ", t1, t2, attach_response=True)
st.remove_response()
st.detrend()
tr_filt = st[0].copy()
tr_filt.filter('bandpass', freqmin=0.1, freqmax=0.45, corners=2, zerophase=True)  # Filter in Secondary Microseisms Frequency Range

# Plot
tr_filt.spectrogram(show=True, cmap='jet', dbscale = True, per_lap = 0.8, wlen=5000)  # Plot Spectrogram